# Inputs

In [1]:
ARTHUR_RULES = {
    # Arthur insists that Lancelot should sit on his right and that Kay should
    # occupy the seat on Arthur's left
    'left': 'Kay',
    'right': 'Lancelot'
}

INCL_RULES = {
    # Bedivere refuses to sit next to anyone but Lionel and Tristan
    'Bedivere': {'Lionel', 'Tristan'}
}

EXCL_RULES = {
    # Gawain won't sit next to Tristan, Lancelot, or Lionel
    'Gawain': {'Tristan', 'Lancelot', 'Lionel'},
    # Gareth won't sit next to Galahad, Lancelot, or Kay
    'Gareth': {'Galahad', 'Lancelot', 'Kay'},
    # Perceval objects to sitting next to Galahad, Lancelot, or, Lionel
    'Perceval': {'Galahad', 'Lancelot', 'Lionel'},
    # Tristan refuses to sit next to Lancelot, Perceval, or Kay
    'Tristan': {'Lancelot', 'Perceval', 'Kay'},
    # Galahad will sit next to anyone except Gawain and Kay
    'Galahad': {'Gawain', 'Kay'},
    # Lionel will not sit next to Galahad
    'Lionel': {'Galahad'}
}

# Helper functions

In [2]:
def get_knights():
    return {
        *ARTHUR_RULES.values(),
        *INCL_RULES,
        *set.union(*INCL_RULES.values()),
        *EXCL_RULES,
        *set.union(*EXCL_RULES.values())
    }

def check_pair(knight_i, knight_j):
    reject_conditions = (
        {knight_i, knight_j}.issubset(ARTHUR_RULES.values()),
        knight_i in EXCL_RULES and knight_j in EXCL_RULES[knight_i],
        knight_j in EXCL_RULES and knight_i in EXCL_RULES[knight_j],
        knight_i in INCL_RULES and knight_j not in INCL_RULES[knight_i],
        knight_j in INCL_RULES and knight_i not in INCL_RULES[knight_j]
    )
    return not any(reject_conditions)

def get_pairs(knights):
    n_knights = len(knights)
    knights_ = sorted(knights)
    return {
        frozenset({knights_[i], knights_[j]})
        for i in range(n_knights - 1)
        for j in range(i + 1, n_knights)
        if check_pair(knights_[i], knights_[j])
    }

def get_rules(knights, pairs):
    return {
        knight: {
            other_knight
            for other_knight in sorted(knights - {knight})
            if {knight, other_knight} in pairs
        }
        for knight in sorted(knights)
    }

# Testing helper functions (not a part of the solution):

In [3]:
knights = get_knights()
knights_ = sorted(knights)
print(f'all knights ({len(knights)}):\n  {knights_}')

pairs = get_pairs(knights)
pairs_ = sorted(sorted(pair) for pair in pairs)
print(f'pairs of knights that might seat next to each other ({len(pairs)}):')
for pair in pairs_:
    print(f'  {pair}')

rules = get_rules(set(knights), pairs)
print('acceptable neighbors for each knight:')
for knight, nbors in rules.items():
    print(f'  {knight} ({len(nbors)}): {sorted(nbors)}')

all knights (9):
  ['Bedivere', 'Galahad', 'Gareth', 'Gawain', 'Kay', 'Lancelot', 'Lionel', 'Perceval', 'Tristan']
pairs of knights that might seat next to each other (14):
  ['Bedivere', 'Lionel']
  ['Bedivere', 'Tristan']
  ['Galahad', 'Lancelot']
  ['Galahad', 'Tristan']
  ['Gareth', 'Gawain']
  ['Gareth', 'Lionel']
  ['Gareth', 'Perceval']
  ['Gareth', 'Tristan']
  ['Gawain', 'Kay']
  ['Gawain', 'Perceval']
  ['Kay', 'Lionel']
  ['Kay', 'Perceval']
  ['Lancelot', 'Lionel']
  ['Lionel', 'Tristan']
acceptable neighbors for each knight:
  Bedivere (2): ['Lionel', 'Tristan']
  Galahad (2): ['Lancelot', 'Tristan']
  Gareth (4): ['Gawain', 'Lionel', 'Perceval', 'Tristan']
  Gawain (3): ['Gareth', 'Kay', 'Perceval']
  Kay (3): ['Gawain', 'Lionel', 'Perceval']
  Lancelot (2): ['Galahad', 'Lionel']
  Lionel (5): ['Bedivere', 'Gareth', 'Kay', 'Lancelot', 'Tristan']
  Perceval (3): ['Gareth', 'Gawain', 'Kay']
  Tristan (4): ['Bedivere', 'Galahad', 'Gareth', 'Lionel']


# Solutions without programming — the elimination approach

This matrix denotes acceptable pairs of knights with the symbol "X":

|              | Arthur   | Bedivere | Galahad  | Gareth   | Gawain   | Kay      | Lancelot | Lionel   | Perceval | Tristan  |
|:------------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|
| **Arthur**   |          |          |          |          |          | X        | X        |          |          |          |
| **Bedivere** |          |          |          |          |          |          |          | X        |          | X        |
| **Galahad**  |          |          |          |          |          |          | X        |          |          | X        |
| **Gareth**   |          |          |          |          | X        |          |          | X        | X        | X        |
| **Gawain**   |          |          |          | X        |          | X        |          |          | X        |          |
| **Kay**      | X        |          |          |          | X        |          |          | X        | X        |          |
| **Lancelot** | X        |          | X        |          |          |          |          | X        |          |          |
| **Lionel**   |          | X        |          | X        |          | X        | X        |          |          | X        |
| **Perceval** |          |          |          | X        | X        | X        |          |          |          |          |
| **Tristan**  |          | X        | X        | X        |          |          |          | X        |          |          |

The matrix is symmetric because being in an acceptable (or unacceptable) pair is a symmetric relation.

This matrix will help us eliminate the pairs of knights that don't want to seat next to each other, i.e. the infeasible arrangements that contain such pairs.

----

The knights of special interest for the elimination approach are **Bedivere** and **Galahad** because each of them has only two acceptable pairs:

* **Bedivere** accepts only and must seat between **Lionel** and **Tristan**
* **Galahad** accepts only and must seat between **Lancelot** and **Tristan**

This also means that **Tristan** must be between **Bedivere** and **Galahad**.

Let's now start with what the king wants. King **Arthur** wants this arrangement:

* `Arthur Lancelot X1 X2 X3 X4 X5 X6 X7 Kay`

where `X1`, `X2`, …, `X7` are unknowns. Of course, the table is round, so **Kay** will be next to (to the left of) **Arthur**.

**Lancelot** is acceptable only to **Galahad** and **Lionel** (and **Kay**, but that's not what **Arthur** wants), so one of them must be at `X1`. Therefore, two possible arrangements are:

1. `Arthur Lancelot Lionel X2 X3 X4 X5 X6 X7 Kay`
1. `Arthur Lancelot Galahad X2 X3 X4 X5 X6 X7 Kay`

The first arrangement (`Arthur Lancelot Lionel X2 X3 X4 X5 X6 X7 Kay`) is infeasible because **Galahad** must seat between **Lancelot** and **Tristan**.

Let's examine the second arrangement (`Arthur Lancelot Galahad X2 X3 X4 X5 X6 X7 Kay`):

* **Galahad** must seat between **Lancelot** and **Tristan**, so it's only possible to have **Tristan** at `X2`:

  * `Arthur Lancelot Galahad Tristan X3 X4 X5 X6 X7 Kay`

* **Bedivere** must be between **Lionel** and **Tristan**, so we must have **Bedivere** at `X3` and **Lionel** at `X4`:

  * `Arthur Lancelot Galahad Tristan Bedivere Lionel X5 X6 X7 Kay`

* The only remaining knight who wants to seat next to **Lionel** is **Gareth** who must get `X5`, while **Gawain** and **Perceval** might take the remaining seats `X6` and `X7`:

  * `Arthur Lancelot Galahad Tristan Bedivere Lionel Gareth Gawain Perceval Kay`
  * `Arthur Lancelot Galahad Tristan Bedivere Lionel Gareth Perceval Gawain Kay`

**These two solutions are _feasible_!**

# Solution with recursive programming

In [4]:
class Solver:
    def __init__(self):
        self.__solutions = []
        self.__knights = get_knights()
        self.__n_knights = len(self.__knights)
        self.__rules = get_rules(self.__knights, get_pairs(self.__knights))

    def _next(self, positions, remain_knights):
        if remain_knights:
            rules = self.__rules
            idx = self.__n_knights - len(remain_knights)
            left_knight, right_knight = positions[idx - 1], positions[idx + 1]
            nbors_for_left_knight = rules[left_knight]
            nbors_for_right_knight = (
                self.__knights if right_knight is None else rules[right_knight]
            )
            allowed_nbors = remain_knights.intersection(
                nbors_for_left_knight, nbors_for_right_knight
            )
            for nbor in allowed_nbors:
                new_positions = positions.copy()
                new_positions[idx] = nbor
                self._next(new_positions, remain_knights - {nbor})
        else:
            self.__solutions.append(positions.copy())

    def solve(self):
        # Start with the positions required by Arthur and continue recursively
        left_knight, right_knight = ARTHUR_RULES['left'], ARTHUR_RULES['right']
        remain_knights = self.__knights - {left_knight, right_knight}
        empty_seats = [None] * len(remain_knights)
        positions = ['Arthur', right_knight, *empty_seats, left_knight]
        self._next(positions, remain_knights)
        return self.__solutions.copy()

In [5]:
result = Solver().solve()

In [6]:
print(f'Possible combinations ({len(result)}):')
for item in result:
    print(f'  {item}')

Possible combinations (2):
  ['Arthur', 'Lancelot', 'Galahad', 'Tristan', 'Bedivere', 'Lionel', 'Gareth', 'Perceval', 'Gawain', 'Kay']
  ['Arthur', 'Lancelot', 'Galahad', 'Tristan', 'Bedivere', 'Lionel', 'Gareth', 'Gawain', 'Perceval', 'Kay']
